In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split
import time
import keras
import matplotlib.pyplot as plt

In [2]:
class PlotValidationLoss(Callback):
    def __init__(self,bin_factor):
        self.bin_factor=bin_factor
        self.val_losses = []

    def on_epoch_end(self, epoch, logs=None):
    
        val_loss = logs.get('val_loss')
        print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")
        self.val_losses.append(val_loss)
        if (epoch+1) % 15 == 0 or epoch == 0:
            self.plot_validation_loss()

    def plot_validation_loss(self):
        epochs_range = range(0, len(self.val_losses))
        plt.plot(epochs_range, self.val_losses, 'b', label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Validation Loss')
        plt.legend()
        plt.savefig('Training-CNN_Validation_Error_'+str(self.bin_factor)+'.png')

In [7]:
start_time = time.time()
import os
from preprocessing import Preprocessing
path_BH = os.path.join('data', 'BH')
path_NS = os.path.join('data', 'NS')
preprocessor = Preprocessing() 

bin_factor=10

BH_powerspectra = preprocessor.collect_all_powerspectra(path_BH, bin_factor=bin_factor, BH=True)
NS_powerspectra = preprocessor.collect_all_powerspectra(path_NS, bin_factor=bin_factor, BH=False)

powerspectra=np.append(np.array(BH_powerspectra),np.array(NS_powerspectra),axis=0)
#data=pd.DataFrame(powerspectra,columns=['freq','power','error','BH?'])

In [52]:
powerspectra[:,:,0:2].reshape(-1,2).shape

(131072, 2)

In [95]:
# post_processing_time = time.time()
X = powerspectra[:,:,0:2].reshape(-1,2)
y = powerspectra[:,:,3].flatten()

# Assuming 'X' is your input data
# Calculate mean and standard deviation
mean = np.mean(X, axis=0)
std_dev = np.std(X, axis=0)
X_standardized = (X - mean) / std_dev

#mean = np.mean(y, axis=0)
#std_dev = np.std(y, axis=0)
#y_standardized = (y - mean) / std_dev
# Standardize the data

# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_standardized, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [71]:
#X_train, X_val, X_test=[data.reshape([8,-1,1]) for data in [X_train,X_val, X_test]]
y_train = keras.utils.to_categorical(y_train, num_classes = 2)
y_val = keras.utils.to_categorical(y_val, num_classes = 2)

In [98]:
X_val.shape

(19661, 2)

In [110]:
# Flatten the input data
#X_train.reshape((-1,30,1))
X_val.flatten().reshape((-1,int(39322/2),1))

array([[[ 4.86232521],
        [ 2.95907641],
        [-0.28471217],
        ...,
        [-0.28471217],
        [-0.32702184],
        [-0.28471217]],

       [[-0.32702184],
        [-0.28471217],
        [-0.32702184],
        ...,
        [-0.32702184],
        [-0.28471217],
        [-0.32702184]]])

In [93]:
X_train.reshape((-1,int(len(X_train)/10),1))

array([[[-0.28471217],
        [-0.32702184],
        [-0.28471217],
        ...,
        [-0.28471217],
        [-0.32702184],
        [-0.28471217]],

       [[-0.32702184],
        [-0.28471217],
        [-0.32702184],
        ...,
        [-0.32702184],
        [-0.28471217],
        [-0.32702184]],

       [[-0.28471217],
        [-0.32702184],
        [-0.28471217],
        ...,
        [-0.28471217],
        [-0.32702184],
        [ 4.13387655]],

       ...,

       [[-0.32702184],
        [-0.28471217],
        [-0.32702184],
        ...,
        [ 2.93300743],
        [-0.28471217],
        [-0.32702184]],

       [[ 5.19786361],
        [ 2.89119314],
        [-0.28471217],
        ...,
        [-0.28471217],
        [-0.32702184],
        [-0.28471217]],

       [[-0.32702184],
        [-0.28471217],
        [-0.32702184],
        ...,
        [-0.32702184],
        [-0.28471217],
        [-0.32702184]]])

In [67]:
X_train.shape,y_train.shape,X_val.shape,y_val.shape

((91750, 2, 1), (91750, 2), (19661, 2, 1), (19661, 2))

In [69]:
# Define your model
model = Sequential()

# Add 1D convolutional layer
model.add(Conv1D(filters=16, kernel_size=128, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])))

# Add max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Add additional convolutional layers or other layers as needed
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the last convolutional layer
model.add(Flatten())

# Add fully connected layers
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization

# Output layer for binary classification
model.add(Dense(2, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d_2. Consider increasing the input size. Received input shape [None, 2, 1] which would produce output shape with a zero or negative value in a dimension.

In [66]:
plot_validation_loss = PlotValidationLoss(bin_factor)
# Train the model
epochs,batch_size=150, 64
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))
#model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

Epoch 1/150


ValueError: in user code:

    File "C:\Users\1926\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\1926\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\1926\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\1926\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\1926\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\1926\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    Va

In [102]:
train_labels.shape

(45875, 2)

In [ ]:
plot_validation_loss = PlotValidationLoss(bin_factor)
# Train the model
epochs,batch_size=150, 64
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

# Make predictions
predictions = model.predict(X_test)

# Optionally, save the model
model.save('CNN_model_'+str(bin_factor)+'.h5')

# Save the test accuracy in a text file
with open('CNN_model_'+str(bin_factor)+'_test_accuracy_and_parameters.txt', 'w') as f:
    f.write(f'Test Accuracy: {test_acc:.4f} \n')
    f.write('____________________________\n')
    f.write('Model Architecture:\n\n')
    model.summary(print_fn=lambda x: f.write(x + '\n'))